In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [2]:
conn = sqlite3.connect("/mnt/c/users/zaned/AppData/Roaming/Godot/app_userdata/BasketballManager/save_data/end_reg_season_test_8/database.db")
conn

In [3]:
df_games = pd.read_sql(
    """
    SELECT ID AS GameID, Home, Away, HomeScore, AwayScore, HomeScore - AwayScore AS Diff
    FROM Games
    WHERE Timestamp >= '2025-0-0-0' AND HomeScore NOT NULL
    """,
    conn,
    index_col="GameID"
)
df_games

,Home,Away,HomeScore,AwayScore,Diff
GameID,,,,,
1,BAY,TEX,38,8,30
2,BAY,KTKY,34,41,-7
3,RICE,BAY,21,43,-22
4,BAY,UNC,29,47,-18
5,GT,CIN,56,71,-15
...,...,...,...,...,...
568,IAST,TCU,78,88,-10
569,UNC,FSU,100,74,26
570,TUL,WICH,36,23,13


In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
ohe = OneHotEncoder()

In [6]:
home_ohe = ohe.fit_transform(df_games[["Home"]].values).toarray()
away_ohe = ohe.transform(df_games[["Away"]].values).toarray()

In [7]:
df_home = pd.DataFrame(home_ohe, columns=ohe.categories_)
df_away = pd.DataFrame(away_ohe, columns=ohe.categories_)
df = df_home - df_away
df.columns = df.columns.get_level_values(0)
df.index = df_games.index
df["Result"] = df_games["Diff"]
df

,ARK,AUB,AZST,BAMA,BAY,BYU,CIN,CLEM,COLO,DUKE,...,UTAH,UVA,VAND,VT,WAST,WICH,WVU,ZAGA,ZONA,Result
GameID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7
3,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-22
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-18
5,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-10
569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26
570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,13


In [8]:
X = df.drop(columns=["Result"])
y = df.Result

In [9]:
linreg = LinearRegression()
linreg.fit(X, y)

LinearRegression()

In [10]:
linreg.intercept_

0.122043560874179

In [11]:
pd.DataFrame(linreg.coef_, index=X.columns, columns=["Rating"]).sort_values(by="Rating", ascending=False)

,Rating
OKLA,36.807689
SCAR,33.259409
KAN,31.840005
OKST,28.910571
TUL,25.040281
ARK,24.651206
UNC,23.780885
IAST,20.898073
HOU,20.592569
TCU,18.538810


In [12]:
# TODO: create-or-replace view in database with this stuff